## 引言 Introduction

1. 背景与现状
医学图像分割主流技术：卷积神经网络（CNN），尤其是全卷积网络（FCN）衍生的 U-Net，凭借对称编码器 - 解码器结构 + 跳跃连接的设计，成为医学图像分割的 “默认选择”，已在 MR 心脏分割、CT 器官分割、结肠镜息肉分割等场景中取得成功。
CNN 的局限性：卷积操作的 “局部性” 导致其难以建模长距离依赖关系，面对患者间纹理、形状、大小差异大的目标结构时，分割性能较弱。
Transformer 的崛起：作为完全依赖注意力机制的架构，Transformer 擅长建模全局上下文，且大规模预训练后迁移能力强，已在 NLP、图像识别任务中达到 / 超越 SOTA，但尚未被系统应用于医学图像分割。
2. 关键问题发现
直接将 Transformer 应用于医学图像分割（图像块 tokenization 编码 + 直接上采样至全分辨率）效果不佳。
根源：Transformer 将输入视为 1D 序列，全程聚焦全局上下文，导致特征分辨率低、缺乏精细定位信息，且直接上采样无法恢复这些信息，最终分割结果粗糙。
3. 解决方案：TransUNet
核心设计：首个融合 CNN 与 Transformer 的医学图像分割框架，兼顾两者优势：
利用 CNN 提取高分辨率、含精细空间细节的低级视觉特征；
利用 Transformer 编码全局上下文信息；
借鉴 U-Net 的 U 形结构，将 Transformer 编码的自注意力特征上采样后，与编码器路径中跳跃连接的高分辨率 CNN 特征融合，实现精确定位。
4. 实验结论
TransUNet 比基于 CNN 的自注意力方法更高效地利用自注意力机制；
深入融合低级特征能进一步提升分割精度；
在多种医学图像分割任务中，性能优于现有竞争方法。

## 方法论 Methodology

### 用Transformer做编码器 Transformer as Encoder

- **图像序列化 Image Sequentialization**: 把 2D 医学图像（如 CT/MR 切片）转 能处理的 1D 序列（类似 换成 TransformerNLP 中把句子拆成单词）。具体操作：
  1. 输入图像：原始图像为$x$，尺寸为$H \times W \times C$。
  2. 分割图像块：将$x$按固定大小$P \times P$(如16x16)分割成不重叠的2D图像块。(patch)
  3. 扁平化：每个$P \times P \times C$的图像块被展平为一个长度为$P^2 \cdot C$的向量。（比如16x16x3的图像块展平为长度768的向量）
  4. 形成序列：所有扁平化的图像块组成一个序列$\{x_p^i\}$，序列长度为$N = \frac{H \times W}{P^2}$。
- **块嵌入 Patch Embedding**: 
  1. 线性映射：使用一个可学习的线性投影矩阵$E \in \mathbb{R}^{(P^2 \cdot C) \times D}$，将每个图像块向量$x_p^i$映射为$D$维嵌入向量$x_p^i E$。
  2. 位置编码：为了保留图像块的空间位置信息，添加可学习的位置编码$E_\text{pos} \in \mathbb{R}^{N \times D}$到嵌入向量中，得到最终的输入序列$z_0 = [x_p^1 E; x_p^2 E; ...; x_p^N E] + E_\text{pos} \in \mathbb{R}^{N \times D}$。
- **Transformer 编码器 Transformer Encoder**: Transformer 编码器由$L$层(多头自注意力层 MSA + 多层感知机 MLP)组成：
  - $z_\ell^\prime = \text{MSA}(\text{LN}(z_{\ell-1})) + z_{\ell-1}$
  - $z_\ell = \text{MLP}(\text{LN}(z_\ell^\prime)) + z_\ell^\prime$

- 为了做语义分割，首先将$z_L \in \mathbb{R}^{N \times D}$重塑为$(\frac{H}{P}, \frac{W}{P}, D)$的张量特征图，然后使用$1\times1$卷积将通道数从$D$转换为$\#\text{cls}$，最后直接通过双线性插值上采样(blinearly up-sample)至原始图像尺寸$(\#\text{cls}, H, W)$。(decoder部分被称为"None")


### TransUNet

<div style="background-color: white; padding: 10px; border-radius: 5px; margin: auto; width:80%; text-align: center;">
    <image src="./assets/transunet_v2.png" />
    <span style="font-size: 12px; color: gray;">图1 TransUNet架构示意图</span>
</div>

- **CNN-Transformer 混合编码器 CNN-Transformer Hybrid as Encoder**: 比起上面直接用 Transformer 作为编码器（直接在原始图像上做 patch embedding），作者提出用 CNN 提取低级视觉特征，再把$1 \times 1$块嵌入后通过Transformer。原因：
    1. 可以在解码器路径中利用中间层的高分辨率CNN特征图；
    2. 混合CNN-Transformer编码器比单纯的Transformer编码器更好。
- **级联上采样器 Cascaded Upsampler**: 引入一种集联上采样器(CUP)
  - 在将隐藏特征$z_L \in \mathbb{R}^{\frac{HW}{P^2} \times D}$重塑为$\frac{H}{P} \times \frac{W}{P} \times D$，我们通过CUP模块逐步上采样至原始分辨率$H \times W$。
  - 其中每个CUP模块包含2个上采样算子，1个$3 \times 3$卷积层和1个ReLU层。

In [2]:
import torch
import torch.nn as nn

# 定义上采样层
in_channels = 512
bilinear = False

if bilinear:  # 使用双线性插值上采样
    # NT: bilinear通过周围4个像素的加权平均值计算新像素值，align_corners保持角点对齐
    up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
else:
    # NT: 转置卷积(反卷积)，能更好保留细节，但参数更多且可能过拟合
    up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)

# 构造输入张量 (B=2, C=512, H=16, W=16)
x = torch.randn(2, in_channels, 16, 16)

# 前向传播
out = up(x)

print(out.shape)  # 输出: torch.Size([2, 512, 32, 32])

torch.Size([2, 256, 32, 32])


In [ ]:
from src.model.unet import UNet
from torchsummary import summary

unet_model = UNet(n_channels=3, n_classes=1)
summary(unet_model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
        DoubleConv-7         [-1, 64, 256, 256]               0
         MaxPool2d-8         [-1, 64, 128, 128]               0
            Conv2d-9        [-1, 128, 128, 128]          73,856
      BatchNorm2d-10        [-1, 128, 128, 128]             256
             ReLU-11        [-1, 128, 128, 128]               0
           Conv2d-12        [-1, 128, 128, 128]         147,584
      BatchNorm2d-13        [-1, 128, 128, 128]             256
             ReLU-14        [-1, 128, 1

In [3]:
from src.model.transunet import TransUnet
from torchsummary import summary

fig_size = (224, 224)
model = TransUnet(
    n_channels=3, 
    n_classes=1,
    embed_dim=768,
    num_heads=8,
    num_layers=4,
    mlp_dim=3072,
    dropout_rate=0.1,
    fig_size=fig_size
)

summary(model, (3, fig_size[0], fig_size[1]))

196
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             864
       BatchNorm2d-2         [-1, 32, 224, 224]              64
              ReLU-3         [-1, 32, 224, 224]               0
            Conv2d-4         [-1, 32, 224, 224]           9,216
       BatchNorm2d-5         [-1, 32, 224, 224]              64
              ReLU-6         [-1, 32, 224, 224]               0
        DoubleConv-7         [-1, 32, 224, 224]               0
         MaxPool2d-8         [-1, 32, 112, 112]               0
            Conv2d-9         [-1, 64, 112, 112]          18,432
      BatchNorm2d-10         [-1, 64, 112, 112]             128
             ReLU-11         [-1, 64, 112, 112]               0
           Conv2d-12         [-1, 64, 112, 112]          36,864
      BatchNorm2d-13         [-1, 64, 112, 112]             128
             ReLU-14         [-1, 6